In [1]:
import pandas as pd


In [4]:
df = pd.read_csv('revolut_stealth_indicatorsNEW.csv')
len(df)

598

In [5]:
exclude_roles = [" hr ", "recruiter", "accountant", "legal", "lawyer"]

exclude_pattern = '|'.join(exclude_roles)  


filtered_df = df[
    ~df['sub_title'].str.contains(exclude_pattern, case=False, na=False)
]

filtered_count = len(filtered_df)

print(f"Исключено ролей: {len(df) - filtered_count}")
print(f"Осталось профилей после фильтрации: {filtered_count}")

Исключено ролей: 7
Осталось профилей после фильтрации: 591


In [6]:
from openai import OpenAI
from typing import Dict, Any
import json
import config

client = OpenAI(api_key=config.OPENAI_API_KEY)

def llm_classifier(sub_title: str, skills: str, model: str) -> Dict[str, Any]:

    prompt = f"""
                Analyze the LinkedIn profile data to detect stealth startups and founder roles. 

                **1. Stealth Startup Indicators**

                a. Direct Evidence (any single mention is enough for `is_stealth = true`):
                    - The word "stealth" (case-insensitive) in the title (e.g., "Stealth Mode," "Stealth Startup")
                    - Terms like "undisclosed," "pre-launch," "unannounced," "confidential," "secret project"
                    - Phrases such as "in stealth" or "in stealth mode"

                b. Indirect Evidence (requires two or more of these signs to set `is_stealth = true`):
                    - No mention of a specific company or organization name (e.g., "Working on AI project" with no company name)
                    - Vague/placeholder descriptions like "New Venture," "Project X," **"Building something new," "building the future,"** 
                        "TBA/TBD project," "stay tuned," "unannounced product," or any synonyms that imply a mysterious or unrevealed project
                    - Very general claims with no real details (e.g., "Building something revolutionary," "Working on a big idea," 
                        "Something exciting coming soon," etc.)
                    - References to "hot" technologies (AI, Blockchain, Web3, Crypto, Quantum, etc.) **without** any concrete context 
                        or company details (indicating a possible stealth R&D effort)

                > **Note**: The model should interpret synonyms of these phrases that indicate a new or not-yet-disclosed project 
                > as potential stealth signals (e.g., "creating the future," "developing something undisclosed," 
                > "launching soon," etc.).

                **2. Founder Role Indicators** 
                - The following **explicit** words/phrases in the title → `is_founder = true`:
                    - "Founder," "Co-founder," "Owner"
                    - "Founding [Role]" (e.g., "Founding Engineer")
                    - References like "Built from scratch," "0 to 1," "my startup"
                - Exclusions (do not count as founder):
                    - "Ex-founder," "Former founder"
                    - "Advisor to startups," "Startup consultant," etc. (an advisory or third-party role, not an active founding member)

                **3. Input Data**:
                - `Current Position`: {sub_title}
                - `Skills`: {skills}

                **4. Output**:
                Return a concise JSON with the following structure:
                ```json
                {{
                    "is_stealth": true or false,
                    "is_founder": true or false,
                    "reason": "short explanation, e.g. 'Stealth in title' or 'No company + vague project'"
                }}
                ```
            """
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "Analyze LinkedIn profiles to identify stealth startups and founder roles."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=100,
            response_format={"type": "json_object"}  
        )
        
        result = json.loads(response.choices[0].message.content)
        return result
        
    except Exception as e:
        print(f"Ошибка при запросе к OpenAI: {e}")
        return {
            "is_stealth": False,
            "is_founder": False,
            "reason": "API Error"
        }

In [8]:
df['is_stealth'] = False  
df['is_founder'] = False  
df['stealth_reason'] = "" 

In [9]:
from tqdm import tqdm

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Обработка профилей"):
    try:
        result = llm_classifier(row['sub_title'], row['skills'], config.gpt_4o)
    except Exception as e:
        print(f"Ошибка при запросе к OpenAI: {e}")
        result = {
            "is_stealth": False,
            "is_founder": False,
            "reason": "API Error"
        }
    
    # Записываем результаты в DataFrame
    df.at[idx, 'is_stealth'] = result['is_stealth']
    df.at[idx, 'is_founder'] = result['is_founder']
    df.at[idx, 'stealth_reason'] = result['reason']

Обработка профилей: 100%|██████████| 598/598 [11:55<00:00,  1.20s/it]


In [20]:
# Фильтруем DataFrame, чтобы получить только строки с is_stealth == True и выбираем столбец sub_title
to_check = df[['is_stealth', 'sub_title', 'li_url']]

# Сохраняем результат в CSV файл
to_check.to_csv('to_check.csv', index=False)

In [21]:
stealth_df = df[df['is_stealth'] == True]

In [23]:
stealth_founder = pd.read_csv('founders_in_stealth_companies.csv')

In [28]:
founder_ids = set(stealth_founder['profile_id'])
new_ids = set(stealth_df['profile_id'])

unique_new_ids = new_ids - founder_ids

profiles_to_check = stealth_df[stealth_df['profile_id'].isin(unique_new_ids)]

print(f"Количество новых профилей для проверки: {len(profiles_to_check)}")


Количество новых профилей для проверки: 16


In [30]:
def company_name_classifier(sub_title: str, model: str) -> Dict[str, Any]:

    prompt = f"""
                Analyze the LinkedIn profile title and determine if it contains a CURRENT company name (True/False).

                Rules for identifying current company names:
                1. Company name should be a specific organization name, not an industry or activity description
                2. Current company names often appear after "@", "at", "in", or similar prepositions
                3. If all companies are prefixed with "ex-", "former", or similar, then there is no current company
                4. Generic descriptions like "stealth", "new venture", "something new", "crypto project" are NOT company names
                5. The company name should be for current employment (not past)

                Examples:

                FALSE cases (no current company mentioned):
                - "Building something new | ex-Google" (only past company)
                - "Something new coming soon" (no company name)
                - "Building something new | ex-Revolut, Lyft, YC S20" (only past companies)
                - "Building something new | Z-Fellow | ex-Yahoo, ex-Revolut" (no current company)
                - "Something New in Crypto (Ex.Revolut, Goldman Sachs)" (industry mention, not company name)
                - "Founder & CEO of Stealth Startup" (generic, not a specific company)
                - "Building the future of fintech" (activity description, not company)
                - "Entrepreneur in Residence" (role without company)

                TRUE cases (current company mentioned):
                - "Senior Product Manager @ KOMI | ex-Spotify & Revolut" (KOMI is current)
                - "Chief of Staff @ Simple App | ex-Revolut" (Simple App is current)
                - "Engineering Lead at Monzo Bank" (Monzo Bank is current)
                - "Product Manager @ N26 | Previously Revolut" (N26 is current)
                - "CEO of TechCorp | ex-Google" (TechCorp is current)

                Input title: "{sub_title}"

                Return JSON format:
                {{
                    "has_current_company": boolean,
                    "reason": "explanation of decision in 5-6 words"
                }}
                """
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "Analyze LinkedIn profiles to identify is there current company name or not."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=100,
            response_format={"type": "json_object"}  
        )
        
        result = json.loads(response.choices[0].message.content)
        return result
        
    except Exception as e:
        print(f"Ошибка при запросе к OpenAI: {e}")
        return {
            "has_current_company": False,
            "reason": "API Error"
        }

In [31]:
profiles_to_check['has_current_company'] = False
profiles_to_check['current_company_reason'] = ""

# Используем tqdm для отображения прогресса
for idx, row in tqdm(profiles_to_check.iterrows(), total=len(profiles_to_check), desc="Анализ профилей"):
    try:
        result = company_name_classifier(row['sub_title'], config.gpt_4o)
        
        # Записываем результаты в DataFrame
        profiles_to_check.at[idx, 'has_current_company'] = result['has_current_company']
        profiles_to_check.at[idx, 'current_company_reason'] = result['reason']
        
    except Exception as e:
        print(f"Ошибка при обработке строки {idx}: {e}")
        profiles_to_check.at[idx, 'current_company_reason'] = f"Error: {str(e)}"

/tmp/ipykernel_90163/788166991.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profiles_to_check['has_current_company'] = False
/tmp/ipykernel_90163/788166991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profiles_to_check['current_company_reason'] = ""
Анализ профилей: 100%|██████████| 16/16 [00:15<00:00,  1.05it/s]


In [34]:
profiles_to_check.value_counts('has_current_company')

has_current_company
False    14
True      2
Name: count, dtype: int64

In [35]:
df_tu_final_classification = profiles_to_check[profiles_to_check['has_current_company'] == False]

In [48]:
import requests
import json
import pandas as pd
from tqdm import tqdm
import os
from typing import Dict, Any
import time

def format_date(date_dict: Dict) -> str:
    if not date_dict or not date_dict.get('year'):
        return None
    
    month = str(date_dict.get('month', 1)).zfill(2)
    year = str(date_dict['year'])
    return f"{year}-{month}"

def extract_profile_info(json_data: Dict) -> Dict[str, Any]:
    try:
        profile_info = {
            'api_sub_title': json_data['sub_title'],
            'current_position': None
        }
        
        if 'position_groups' in json_data and json_data['position_groups']:
            current_group = json_data['position_groups'][0]
            if current_group['profile_positions']:
                current_position = current_group['profile_positions'][0]
                profile_info['current_position'] = {
                    'company': current_group['company']['name'],
                    'title': current_position['title'],
                    'start_date': format_date(current_position['date']['start']),
                    'end_date': format_date(current_position['date']['end']),
                    'employment_type': current_position['employment_type'],
                    'location': current_position.get('location', 'N/A')
                }
        
        return profile_info
    
    except Exception as e:
        print(f"Ошибка при обработке данных: {str(e)}")
        return None

def process_profiles(df_input: pd.DataFrame):
    os.makedirs('final_request_to_api', exist_ok=True)
    
    new_data = []
    
    
    url = "https://api.proapis.com/iscraper/v4/profile-details"
    headers = {
        "Content-Type": "application/json",
        "X-Api-Key": config.PRO_API_KEY
    }
    
    
    for _, row in tqdm(df_input.iterrows(), total=len(df_input), desc="Обработка профилей"):
        try:
            
            payload = {"profile_id": row['profile_id']}
            response = requests.post(url, headers=headers, json=payload)
            
            if response.status_code == 200:
                json_data = response.json()
                json_file_path = f"final_request_to_api/{row['profile_id']}.json"
                with open(json_file_path, 'w', encoding='utf-8') as f:
                    json.dump(json_data, f, indent=4, ensure_ascii=False)
                
                profile_info = extract_profile_info(json_data)
                
                if profile_info:
                    profile_data = {
                        'profile_id': row['profile_id'],
                        'first_name': row['first_name'],
                        'last_name': row['last_name'],
                        'linkedin_sub_title': row['sub_title'],
                        'li_url': row['li_url'],
                        'api_sub_title': profile_info['api_sub_title']
                    }
                    
                    if profile_info['current_position']:
                        current = profile_info['current_position']
                        profile_data.update({
                            'current_company': current['company'],
                            'current_title': current['title'],
                            'start_date': current['start_date'],
                            'end_date': current['end_date'],
                            'employment_type': current['employment_type'],
                            'location': current['location']
                        })
                    else:
                        profile_data.update({
                            'current_company': None,
                            'current_title': None,
                            'start_date': None,
                            'end_date': None,
                            'employment_type': None,
                            'location': None
                        })
                    
                    new_data.append(profile_data)
            
            else:
                print(f"Ошибка API для {row['profile_id']}: {response.status_code}")
            
            time.sleep(1)
            
        except Exception as e:
            print(f"Ошибка при обработке профиля {row['profile_id']}: {str(e)}")
    
    new_df = pd.DataFrame(new_data)
    new_df.to_csv('profiles_with_current_positions.csv', index=False)
    print("\nДанные сохранены в 'profiles_with_current_positions.csv'")
    
    return new_df

result_df = process_profiles(df_tu_final_classification)

Обработка профилей:  36%|███▌      | 5/14 [00:07<00:14,  1.63s/it]

Ошибка API для david-kfm: 404


Обработка профилей: 100%|██████████| 14/14 [00:22<00:00,  1.58s/it]


Данные сохранены в 'profiles_with_current_positions.csv'


In [50]:
rows_to_add = []

for idx, row in result_df.iterrows():
    try:
        if 'stealth' in row['current_company'].lower():
            rows_to_add.append(row)
        elif 'revolut' in row['current_company'].lower() and row['end_date'] != None:
            rows_to_add.append(row)
    except:
        pass

df_stealth_companies = pd.DataFrame(rows_to_add)
df_stealth_companies

,profile_id,first_name,last_name,linkedin_sub_title,li_url,api_sub_title,current_company,current_title,start_date,end_date,employment_type,location
1,mariozaharioudakis,Mario,Z.,Working on something 👌,https://www.linkedin.com/in/mariozaharioudakis/,Working on something 👌,Revolut,CEO Office - Founder’s Associate,2021-None,2023-None,Full-time,"New York, New York, United States"
9,carl-fernandes-a738296a,Carl,Fernandes,"Building something new | ex-Revolut, Lyft, YC S20",https://www.linkedin.com/in/carl-fernandes-a73...,"Building something new | ex-Revolut, Lyft, YC S20",Revolut,"Head of Product, Insurance",2022-03,2024-03,Full-time,"Boston, Massachusetts, United States"


In [51]:
for li_url in df_stealth_companies['li_url']:
    print(li_url)

https://www.linkedin.com/in/mariozaharioudakis/
https://www.linkedin.com/in/carl-fernandes-a738296a/


In [53]:
fdf = pd.read_csv('founders_in_stealth_companies.csv')
len(fdf)

19

In [60]:
fdf['end_date'] = None

In [62]:
df_combined = pd.concat([fdf, df_stealth_companies], ignore_index=True)

In [64]:
df_combined.columns

Index(['profile_id', 'first_name', 'last_name', 'linkedin_sub_title', 'li_url',
       'api_sub_title', 'current_company', 'current_title', 'start_date',
       'employment_type', 'location', 'end_date'],
      dtype='object')

In [65]:
df_combined = df_combined[['first_name', 'last_name', 'linkedin_sub_title', 'li_url']]

In [69]:
df_combined.to_csv('founders_in_stealth_companies_final.csv', index=False)